# Agent dynamic traits
This notebook provides a demonstration of the optional `Agent` parameter `dynamic_traits_function` that can be used to generate agent traits dynamically based on the question being asked or the scenario in which the question is asked. 

Learn more about this method in the docs: [Agent dynamic traits function](https://docs.expectedparrot.com/en/latest/agents.html#dynamic-traits-function)

## How it works
Agents are created by passing a dictionary of `traits` to an `Agent` object. For example:

In [1]:
from edsl import Agent

my_agent = Agent(
    traits={"persona": "You are a middle-aged mom.", "current_vehicle": "minivan"}
)

When we run a survey with this agent, the language model will reference the agent's traits in generating responses. We can test this:

In [2]:
from edsl.questions import QuestionFreeText
from edsl import Survey

q1 = QuestionFreeText(question_name="age", question_text="How old are you?")
q2 = QuestionFreeText(question_name="car", question_text="What are you driving?")
survey = Survey([q1, q2])

results = survey.by(my_agent).run()

results.select("answer.*").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                 ┃ answer                                                 ┃
┃ .car                                                   ┃ .age                                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Oh, I drive a minivan. It's perfect for shuttling the  │ Oh, I'm in my mid-40s. Time sure does fly, doesn't it? │
│ kids around and running errands. Plus, there's plenty  │                                                        │
│ of room for groceries and all their sports gear!       │                                                        │
└────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────┘

## Designing question-based traits
For efficiency or other reasons, we may want to minimize the set of traits that we pass to the agent when we create it, and instead only generate certain traits to use with specific questions. To do this, we can create a method for the desired conditional logic and pass it to an agent as the `dynamic_traits_function` parameter:

In [3]:
# Create a method for the desired logic
def favorite_color_response(question):
    if question.question_name == "favorite_color":
        return {
            "favorite_color": "I'd prefer not to say."
        }  # trait to be passed to the agent


# Pass it to the agent
a = Agent(dynamic_traits_function=favorite_color_response)

# Test it by running a relevant question
q = QuestionFreeText(
    question_name="favorite_color", question_text="What is your favorite color?"
)

results = q.by(a).run()

# Inspect the response
results.select("favorite_color").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                 ┃
┃ .favorite_color        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━┩
│ I'd prefer not to say. │
└────────────────────────┘